# Single Molecule Fluorescence Photobleaching Data Analyses—Step Finding Algorithm

In [ ]:
pip install tqdm

In [ ]:
import pickle
import numpy as np
import pandas as pd
import ruptures as rpt
from itertools import product
from collections import Counter
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

from matplotlib.backends.backend_pdf import PdfPages as pltPdf

In [ ]:
os.getcwd()

### Import Sample Data

In [ ]:
fname = '~/path/to/data'
signal = pd.read_csv(fname)
signal.columns = ['ts', 'val']
signal.head()

In [ ]:
handle = fname.split("\\")[-1]
handle = handle.split(".")[0]
handle

# fname = fname
handle = fname.split(".")[0]
signal = pd.read_csv(fname)
signal.columns = ['ts', 'val']
signal.head()

In [ ]:
signal.tail()

In [ ]:
# inspect input signal
plt.rcParams["figure.figsize"] = (20,4)
plt.plot(signal.ts, signal.val, label='Signal')
plt.title('Photon Intensity by Elapsed Time')
plt.xlabel('Time (sec)')
plt.ylabel('Intensity (unit/sec)')
plt.margins(0)

plt.savefig('{}_input_signal.svg'.format(handle), format='svg', dpi=1600)

plt.show()
plt.close()

#### Observations

A major inflection point is seen at about 10 secs. Changepoint detection leaving out the rest of the data makes sense.

## Select most frequent Changepoints from all the methods and models

Using entire signal length

In [ ]:
%%time

plt.rcParams["figure.figsize"] = (20,4)
plt.ioff()  # interactive mode off, use plt.show() to force inline display of plots

MAX_BREAKPOINTS = 15
BREAKPOINTS = range(4, MAX_BREAKPOINTS+1)

# input signal
X = signal.val.values

# container for experiment results
res_ls = list()

# container for all valid change points (for freq analysis)
cps_ls = list()

### WINDOW METHOD
method = "Window"

# training parameters
MODELS = ["rbf", "l1",]
WINDOW_SIZES = [8, 10, 12, 15, 18, 20, 22, 25]

# container for multi-page pdf plot
pdfFig = pltPdf("{}_cps_plots_{}.pdf".format(handle, method))

# train detector
for (m, w, b) in tqdm(product(MODELS, WINDOW_SIZES, BREAKPOINTS)):
    cps = rpt.Window(width=w, min_size=2, jump=1).fit_predict(X, n_bkps=b)
    # store changepoints
    cps_ls.extend(cps[:-1])
    # plot changepoints
    fig, ax = rpt.display(X, cps)
    fig.suptitle("Method: {}, Model: {}, BreakPoints: {}".format(method, m, b))
    pdfFig.savefig(fig)
    plt.close(fig)

pdfFig.close()

### PELT METHOD
method = "Pelt"

# training parameters
MODELS = ["rbf", ]
PENALTIES = np.linspace(0.25, 2.5, 20)

# container for multi-page pdf plot
pdfFig = pltPdf("{}_cps_plots_{}.pdf".format(handle, method))

# train detector
for (m, p) in tqdm(product(MODELS, PENALTIES)):
    cps = rpt.Pelt(model=m, min_size=2, jump=1).fit_predict(X, pen=p)
    # store changepoints
    cps_ls.extend(cps[:-1])
    # plot changepoints
    fig, ax = rpt.display(X, cps)
    fig.suptitle("Method: {}, Model: {}, Penalty: {}".format(method, m, p))
    pdfFig.savefig(fig)
    plt.close(fig)

pdfFig.close()

### OTHER METHODS
METHODS = ["Binseg", "BottomUp", "Dynp"]

# training parameters
MODELS = ["rbf", "l1"]

for method in tqdm(METHODS):
    # container for multi-page pdf plot
    pdfFig = pltPdf("{}_cps_plots_{}.pdf".format(handle, method))
    
    # train detector
    for (m, b) in product(MODELS, BREAKPOINTS):
        if method == "Binseg":
            cps = rpt.Binseg(model=m, min_size=2, jump=1).fit_predict(X, n_bkps=b)
        if method == "BottomUp":
            cps = rpt.BottomUp(model=m, min_size=2, jump=1).fit_predict(X, n_bkps=b)
        if method == "Dynp":
            cps = rpt.Dynp(model=m, min_size=2, jump=1).fit_predict(X, n_bkps=b)            
        # store changepoints
        cps_ls.extend(cps[:-1])            
        # plot changepoints
        fig, ax = rpt.display(X, cps)
        fig.suptitle("Method: {}, Model: {}, BreakPoints: {}".format(method, m, b))
        pdfFig.savefig(fig)
        plt.close(fig)
    pdfFig.close()
    
cps_fq = Counter(cps_ls)  # produces dict ordered by counts of cp occurances

"""
# archive changepoints, and their count by occurance
with open("cps_list.pkl", "wb") as f:
    pickle.dump(cps, f)
with open("cps_freq.pkl", "wb") as f:
    pickle.dump(cps_fq, f)
"""

In [ ]:
print(cps_fq)

In [ ]:
MAX_BREAKPOINTS = 15
top_n = dict(cps_fq.most_common(MAX_BREAKPOINTS))
top_n = sorted(list(top_n.keys())) + [len(signal.ts)]
scale = max(cps_fq.values())
top_n, scale

In [ ]:
%%time

top_n = dict(cps_fq.most_common(MAX_BREAKPOINTS))

top_n = sorted(list(top_n.keys())) + [len(signal.ts)-2]

scale = max(cps_fq.values())

results_ls = list()

prev = 0

for ix in top_n:
    results_ls.append([ix, signal.ts[ix],
                       signal.val[ix-1], signal.val[ix], signal.val[ix+1], 
                       cps_fq[ix]/scale, np.mean(signal.val[prev:ix])])
    prev = ix

results_df = pd.DataFrame(results_ls, columns=["Id", "Timestamp", "Prev Signal Value",
                                               "Changepoint Value", "Next Signal Value",
                                               "Changepoint Significance", "Average Signal Over Window"
                                              ])
results_df

In [ ]:
results_df.to_csv("{}_final_changepoints.csv".format(handle))

### Plot Changepoints

In [ ]:
%%time

pdfFig = pltPdf("{}_final_signal_with_cps_plots.pdf".format(handle))

for n in BREAKPOINTS:
    # select top N changepoints from the ordered Dict
    top_n = dict(cps_fq.most_common(n))
    print(n, top_n)
    
    # plot parameters
    plt.rcParams["figure.figsize"] = (30, 4)
    plt.title('Significant {} Change-Points: Photon Intensity by Elapsed Time'.format(n))
    plt.xlabel('Time (sec)')
    plt.ylabel('Intensity (unit/sec)')
    plt.margins(0)

    # plot the original signal
    plt.plot(signal.ts, signal.val, label='Signal')
    
    # overlay averaged signal values
    overlay = list()
    prev = 0
    for ix in sorted(list(top_n.keys())):
        plt.axvline(x=signal.ts[ix], color='g')  # plot changepoints
        overlay.extend([np.mean(signal.val[prev:ix])]*(ix-prev))
        prev = ix
    overlay.extend([np.mean(signal.val[ix:])]*(len(signal.val)-ix))
    
    # plot overlay values
    plt.plot(signal.ts, overlay, color='k', label='Averaged Signal')

    # save plot
    #plt.savefig('signal_with_{}_cps.svg'.format(n), format='svg', dpi=1600)
    
    pdfFig.savefig()
    plt.close()

pdfFig.close()

### PROCESSING COMPLETE

In [ ]:
pip install ruptures

In [ ]:
cps_fq.most_common(1)

In [ ]:
results_df

In [ ]:
excelOut = results_df[["Timestamp", "Changepoint Value"]]
excelOut
excelOut.to_csv("timeStampAndChangePointData.csv")


In [ ]:
import seaborn as sns

x=excelOut[["Timestamp"]]
y=excelOut[["Changepoint Value"]]
sns.lineplot(data = excelOut, x = "Timestamp", y = "Changepoint Value")

In [ ]:
pip install seaborn

In [ ]:
signal.ts

In [ ]:
overlay

In [ ]:
plt.plot(signal.ts, overlay)

In [ ]:
excelSpread = pd.DataFrame({'signal':signal.ts, 'overlay':overlay})


In [ ]:
excelSpread.to_csv("smoothData.csv")
